In [18]:
import multiprocessing
from multiprocessing import sharedctypes
import pandas as pd
import glob
import string
from collections import defaultdict
import tqdm
import numpy as np
from collections import defaultdict

import pyrosetta


In [2]:
pyrosetta.init()

PyRosetta-4 2019 [Rosetta PyRosetta4.conda.linux.CentOS.python37.Release 2019.19.post.dev+17.commits.d5a7d8129801bf7eaf383dba4ad48c78085ffd42 2019-05-20T01:48:51] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.conda.linux.CentOS.python37.Release r12571 2019.19.post.dev+17.commits.d5a7d81 d5a7d8129801bf7eaf383dba4ad48c78085ffd42 http://www.pyrosetta.org 2019-05-20T01:48:51
core.init: command: PyRosetta -ex1 -ex2aro -database /home/rdkibler/.conda/envs/pyro/lib/python3.7/site-packages/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=1828616719 seed_offset=0 real_seed=1828616719
core.init.random: RandomGenerator:init: Normal mode, seed=1828616719 RG_type=mt19937


In [3]:
design_paths = glob.glob("../output/*/*.pdb")

In [4]:
rs_core = pyrosetta.rosetta.core.select.residue_selector.LayerSelector()
rs_core.set_layers(pick_core=True, pick_boundary=False, pick_surface=False)
rs_core.set_use_sc_neighbors(False)
rs_core.set_ball_radius(2.5)
rs_core.set_cutoffs(core=20,surf=35)
rs_hbnet = pyrosetta.rosetta.core.select.residue_selector.ResiduePDBInfoHasLabelSelector()
rs_hbnet.set_label("HBNet")
rs_and = pyrosetta.rosetta.core.select.residue_selector.AndResidueSelector()
rs_and.add_residue_selector(rs_core)
rs_and.add_residue_selector(rs_hbnet)
rs_buried_hbnets = rs_and

def find_buried_hbnets(pdb_path):
    pose = pyrosetta.pose_from_file(pdb_path)
    rs = np.array(rs_buried_hbnets.apply(pose))
    return (pdb_path, ",".join([f"{('X','A','B')[pose.residue(resi + 1).chain()]}_{pose.residue(resi + 1).name1()}_{resi + 1}" for resi, selected in enumerate(rs) if selected]))

core.select.residue_selector.LayerSelector: Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: Setting core=true boundary=false surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: Setting LayerSelector to use rolling ball-based occlusion to determine burial.
core.select.residue_selector.LayerSelector: Setting radius for rolling ball algorithm to 2.5 in LayerSelector.  (Note that this will have no effect if the sidechain neighbors method is used.)
core.select.residue_selector.LayerSelector: Set cutoffs for core and surface to 20 and 35, respectively, in LayerSelector.


In [5]:
find_buried_hbnets(design_paths[0])

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 979 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 2.06724 seconds.
core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm_00638/churro_9x25GB28GB_2_works_dimer_nm_00638_hb_0001.pdb' automatically determined to be of type PDB


('../output/churro_9x25GB28GB_2_works_dimer_nm_00638/churro_9x25GB28GB_2_works_dimer_nm_00638_hb_0001.pdb',
 'A_S_14,A_S_17,B_D_468,B_T_494')

In [6]:
print(f"reading {len(design_paths)} designs")

agents = 16
chunksize = 1


pool = multiprocessing.Pool(processes=agents)
buried_hbnets = [x for x in tqdm.tqdm_notebook(pool.imap_unordered(find_buried_hbnets, design_paths),total=len(design_paths))]
pool.close()
pool.join()

reading 12954 designs


/home/rdkibler/.conda/envs/pyro/lib/python3.7/site-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm_00638/churro_9x25GB28GB_2_works_dimer_nm_00638_hb_0001.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0010.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0011.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0017.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0019.pdb' automatically determined to be of type PDB
core.import_pose.import_pose: File '../output/churro_9x25GB28GB_2_works_dimer_nm

In [7]:
buried_hbnets

[('../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0010.pdb',
  'A_S_17,B_D_468,B_S_471,B_S_494'),
 ('../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0011.pdb',
  'A_N_10,B_T_468,B_S_497,B_S_501'),
 ('../output/churro_9x25GB28GB_2_works_dimer_nm_00638/churro_9x25GB28GB_2_works_dimer_nm_00638_hb_0001.pdb',
  'A_S_14,A_S_17,B_D_468,B_T_494'),
 ('../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0016.pdb',
  'A_D_3,A_T_10,A_S_17,A_Q_18,A_Q_37,B_N_460,B_S_461,B_N_464,B_D_468,B_S_494,B_T_501'),
 ('../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0017.pdb',
  'A_D_3,A_S_10,A_T_17,B_N_460,B_S_461,B_N_464,B_D_468,B_S_471,B_S_494,B_T_501'),
 ('../output/churro_9x25GB28GB_2_works_dimer_nm_00735/churro_9x25GB28GB_2_works_dimer_nm_00735_hb_0023.pdb',
  'A_D_3,A_T_10,A_S_17,A_Q_18,A_Q_37,B_N_460,B_S_461,B_N_464,B_D_4

In [10]:
filename = "buried_nets.txt"
with open(filename, 'w') as f:
    for path, net in buried_hbnets:
        f.write(f"{path} {net}\n")

In [12]:
networks = [net for path, net in buried_hbnets]

In [27]:
uniq_nets = defaultdict(lambda: 0)
for net in networks:
    uniq_nets[net] += 1
for x in sorted(uniq_nets.items(), key=lambda x:x[1], reverse=True):
    print(f"{x[1]}\t{x[0]}" )

169	A_S_14,A_T_17,A_Q_18,A_Q_37,B_S_494
134	A_S_14,A_S_17,A_Q_18,A_Q_37,B_S_494
116	A_S_14,A_T_17,A_S_18,A_K_37,B_S_494
112	A_S_14,A_S_17,B_D_468,B_T_494
74	A_S_14,A_T_17,A_T_18,A_K_37,B_S_494
71	A_S_14,A_S_17,B_D_468,B_S_494
69	A_S_14,A_S_17,A_S_18,A_K_37,B_S_494
58	A_S_14,A_T_17,B_D_468,B_T_494
39	A_S_17,B_D_468,B_S_471,B_S_494
37	A_S_14,A_S_17,A_T_18,A_K_37,B_S_494
32	A_S_14,A_S_17,A_K_37,B_D_468,B_T_494
32	A_S_14,A_S_17,A_K_37,B_D_468,B_S_494
30	A_N_10,A_S_14,B_S_464,B_S_468,B_S_497
30	A_S_14,A_Q_30,A_S_34,A_K_37,B_S_494
24	A_S_17,A_N_21,B_S_471,B_S_490
23	A_S_14,A_T_17,A_Q_30,A_S_34,A_K_37,B_S_494
23	A_S_14,A_T_17,B_S_468,B_T_494
22	A_T_17,A_Q_18,A_Q_37,B_S_494,B_S_498
22	A_N_7,A_S_10,B_N_461,B_S_501
22	A_S_14,A_T_17,A_T_18,A_K_37,B_T_494
21	A_N_21,B_S_471,B_S_475,B_S_487,B_S_490
21	A_N_21,B_S_471,B_S_475,B_S_490
21	A_S_14,A_T_17,B_D_468,B_S_494
21	A_D_7,B_E_457,B_R_460,B_T_461,B_D_464
20	A_S_14,A_Q_18,A_Q_37,B_S_494,B_S_498
20	A_S_14,A_T_17,A_Q_18,A_Q_37,B_S_494,B_S_498
20	A_N_18

In [28]:
biggest_net_path = None
biggest_net_size = 0

for path,net in buried_hbnets:
    size = len(net.split(","))
    if biggest_net_size < size:
        print(f"{net} is bigger!")
        biggest_net_size = size
        biggest_net_path = path

A_S_17,B_D_468,B_S_471,B_S_494 is bigger!
A_D_3,A_T_10,A_S_17,A_Q_18,A_Q_37,B_N_460,B_S_461,B_N_464,B_D_468,B_S_494,B_T_501 is bigger!
A_D_3,A_T_10,A_S_17,A_Q_18,A_R_30,A_Q_37,B_N_460,B_S_461,B_N_464,B_D_468,B_S_494,B_T_501 is bigger!
A_N_8,A_N_10,A_Q_18,A_N_21,A_S_30,A_K_37,A_S_45,A_S_67,B_S_448,B_T_467,B_T_468,B_S_487,B_E_491,B_S_497,B_S_498,B_S_501,B_S_505 is bigger!
A_N_7,A_N_8,A_Q_18,A_N_21,A_S_30,A_K_37,A_S_45,A_S_67,B_E_448,B_E_457,B_R_460,B_T_461,B_T_464,B_S_475,B_S_487,B_E_491,B_E_493,B_S_498,B_S_505 is bigger!
A_N_3,A_S_7,A_D_8,A_S_10,A_Q_11,A_S_14,A_S_17,A_S_18,A_K_37,A_R_48,A_S_64,B_S_448,B_S_457,B_E_461,B_N_464,B_D_487,B_S_494,B_D_500,B_S_501,B_N_504,B_S_505,B_D_508 is bigger!


In [29]:
biggest_net_path

'../output/churro_9x25GB28GB_2_works_dimer_nm_00378/churro_9x25GB28GB_2_works_dimer_nm_00378_hb_0055.pdb'